# Labelme转YOLO-Keypoint-批量

同济子豪兄 2023-4-17

## 导入工具包

In [1]:
import os
import json
import shutil
import numpy as np
from tqdm import tqdm

## 删除系统自动生成的多余文件

### 查看待删除的多余文件

In [2]:
!find . -iname '__MACOSX'

In [3]:
!find . -iname '.DS_Store'

In [4]:
!find . -iname '.ipynb_checkpoints'

./【C】Labelme转YOLO-Keypoint-单个文件/.ipynb_checkpoints
./.ipynb_checkpoints


### 删除多余文件

In [5]:
!for i in `find . -iname '__MACOSX'`; do rm -rf $i;done

In [6]:
!for i in `find . -iname '.DS_Store'`; do rm -rf $i;done

In [7]:
!for i in `find . -iname '.ipynb_checkpoints'`; do rm -rf $i;done

### 验证多余文件已删除

In [8]:
!find . -iname '__MACOSX'

In [9]:
!find . -iname '.DS_Store'

In [10]:
!find . -iname '.ipynb_checkpoints'

## 数据集类别信息

In [11]:
Dataset_root = 'SJB_25_Dataset'

In [12]:
# 框的类别
bbox_class = {
    'sjb_rect':0  
}

In [13]:
# 关键点的类别
keypoint_class = ['angle_30', 'angle_60', 'angle_90']

In [14]:
os.chdir(Dataset_root)

In [15]:
os.mkdir('labels')
os.mkdir('labels/train')
os.mkdir('labels/val')

## 函数-处理单个labelme标注json文件

In [16]:
def process_single_json(labelme_path, save_folder='../../labels/train'):
    
    with open(labelme_path, 'r', encoding='utf-8') as f:
        labelme = json.load(f)

    img_width = labelme['imageWidth']   # 图像宽度
    img_height = labelme['imageHeight'] # 图像高度

    # 生成 YOLO 格式的 txt 文件
    suffix = labelme_path.split('.')[-2]
    yolo_txt_path = suffix + '.txt'

    with open(yolo_txt_path, 'w', encoding='utf-8') as f:

        for each_ann in labelme['shapes']: # 遍历每个标注

            if each_ann['shape_type'] == 'rectangle': # 每个框，在 txt 里写一行

                yolo_str = ''

                ## 框的信息
                # 框的类别 ID
                bbox_class_id = bbox_class[each_ann['label']]
                yolo_str += '{} '.format(bbox_class_id)
                # 左上角和右下角的 XY 像素坐标
                bbox_top_left_x = int(min(each_ann['points'][0][0], each_ann['points'][1][0]))
                bbox_bottom_right_x = int(max(each_ann['points'][0][0], each_ann['points'][1][0]))
                bbox_top_left_y = int(min(each_ann['points'][0][1], each_ann['points'][1][1]))
                bbox_bottom_right_y = int(max(each_ann['points'][0][1], each_ann['points'][1][1]))
                # 框中心点的 XY 像素坐标
                bbox_center_x = int((bbox_top_left_x + bbox_bottom_right_x) / 2)
                bbox_center_y = int((bbox_top_left_y + bbox_bottom_right_y) / 2)
                # 框宽度
                bbox_width = bbox_bottom_right_x - bbox_top_left_x
                # 框高度
                bbox_height = bbox_bottom_right_y - bbox_top_left_y
                # 框中心点归一化坐标
                bbox_center_x_norm = bbox_center_x / img_width
                bbox_center_y_norm = bbox_center_y / img_height
                # 框归一化宽度
                bbox_width_norm = bbox_width / img_width
                # 框归一化高度
                bbox_height_norm = bbox_height / img_height

                yolo_str += '{:.5f} {:.5f} {:.5f} {:.5f} '.format(bbox_center_x_norm, bbox_center_y_norm, bbox_width_norm, bbox_height_norm)

                ## 找到该框中所有关键点，存在字典 bbox_keypoints_dict 中
                bbox_keypoints_dict = {}
                for each_ann in labelme['shapes']: # 遍历所有标注
                    if each_ann['shape_type'] == 'point': # 筛选出关键点标注
                        # 关键点XY坐标、类别
                        x = int(each_ann['points'][0][0])
                        y = int(each_ann['points'][0][1])
                        label = each_ann['label']
                        if (x>bbox_top_left_x) & (x<bbox_bottom_right_x) & (y<bbox_bottom_right_y) & (y>bbox_top_left_y): # 筛选出在该个体框中的关键点
                            bbox_keypoints_dict[label] = [x, y]

                ## 把关键点按顺序排好
                for each_class in keypoint_class: # 遍历每一类关键点
                    if each_class in bbox_keypoints_dict:
                        keypoint_x_norm = bbox_keypoints_dict[each_class][0] / img_width
                        keypoint_y_norm = bbox_keypoints_dict[each_class][1] / img_height
                        yolo_str += '{:.5f} {:.5f} {} '.format(keypoint_x_norm, keypoint_y_norm, 2) # 2-可见不遮挡 1-遮挡 0-没有点
                    else: # 不存在的点，一律为0
                        yolo_str += '0 0 0 '
                # 写入 txt 文件中
                f.write(yolo_str + '\n')
                
    shutil.move(yolo_txt_path, save_folder)
    print('{} --> {} 转换完成'.format(labelme_path, yolo_txt_path))    

## 转换训练集标注文件至`labels/train`目录

In [17]:
os.chdir('labelme_jsons/train')

In [18]:
save_folder = '../../labels/train'
for labelme_path in os.listdir():
    try:
        process_single_json(labelme_path, save_folder=save_folder)
    except:
        print('******有误******', labelme_path)
print('YOLO格式的txt标注文件已保存至 ', save_folder)

DSC_0269.json --> DSC_0269.txt 转换完成
MVIMG_20230331_080915.json --> MVIMG_20230331_080915.txt 转换完成
DSC_0289.json --> DSC_0289.txt 转换完成
DSC_0282.json --> DSC_0282.txt 转换完成
DSC_0260.json --> DSC_0260.txt 转换完成
DSC_0278.json --> DSC_0278.txt 转换完成
MVIMG_20230331_080914.json --> MVIMG_20230331_080914.txt 转换完成
DSC_0301.json --> DSC_0301.txt 转换完成
DSC_0219.json --> DSC_0219.txt 转换完成
DSC_0274.json --> DSC_0274.txt 转换完成
DSC_0259.json --> DSC_0259.txt 转换完成
DSC_0280.json --> DSC_0280.txt 转换完成
DSC_0240.json --> DSC_0240.txt 转换完成
MVIMG_20230331_080912.json --> MVIMG_20230331_080912.txt 转换完成
DSC_0293.json --> DSC_0293.txt 转换完成
DSC_0265.json --> DSC_0265.txt 转换完成
DSC_0283.json --> DSC_0283.txt 转换完成
DSC_0236.json --> DSC_0236.txt 转换完成
DSC_0285.json --> DSC_0285.txt 转换完成
DSC_0209.json --> DSC_0209.txt 转换完成
YOLO格式的txt标注文件已保存至  ../../labels/train


In [19]:
os.chdir('../../')

## 转换测试集标注文件至`labels/val`目录

In [20]:
os.chdir('labelme_jsons/val')

In [21]:
save_folder = '../../labels/val'
for labelme_path in os.listdir():
    try:
        process_single_json(labelme_path, save_folder=save_folder)
    except:
        print('******有误******', labelme_path)
print('YOLO格式的txt标注文件已保存至 ', save_folder)

DSC_0245.json --> DSC_0245.txt 转换完成
MVIMG_20230331_080908.json --> MVIMG_20230331_080908.txt 转换完成
DSC_0281.json --> DSC_0281.txt 转换完成
DSC_0284.json --> DSC_0284.txt 转换完成
MVIMG_20230331_080920.json --> MVIMG_20230331_080920.txt 转换完成
YOLO格式的txt标注文件已保存至  ../../labels/val


In [22]:
os.chdir('../../')

## 删除labelme格式的标注文件

In [23]:
!rm -rf labelme_jsons

In [24]:
os.chdir('../')

## 删除系统自动生成的多余文件

### 查看待删除的多余文件

In [25]:
!find . -iname '__MACOSX'

In [26]:
!find . -iname '.DS_Store'

In [27]:
!find . -iname '.ipynb_checkpoints'

### 删除多余文件

In [28]:
!for i in `find . -iname '__MACOSX'`; do rm -rf $i;done

In [29]:
!for i in `find . -iname '.DS_Store'`; do rm -rf $i;done

In [30]:
!for i in `find . -iname '.ipynb_checkpoints'`; do rm -rf $i;done

### 验证多余文件已删除

In [31]:
!find . -iname '__MACOSX'

In [32]:
!find . -iname '.DS_Store'

In [33]:
!find . -iname '.ipynb_checkpoints'

## 得到YOLO格式的关键点检测数据集

## 查看数据集目录结构

In [34]:
# !pip install seedir emoji

In [35]:
import seedir as sd
sd.seedir('SJB_25_Dataset', style='emoji', depthlimit=2)

📁 SJB_25_Dataset/
├─📁 images/
│ ├─📁 train/
│ └─📁 val/
└─📁 labels/
  ├─📁 train/
  └─📁 val/
